## Classification
- **Culturally Exclusive**: 'caponata' -> Italy
- **Cultural Representative**: 'pizza' -> Italy
- **Cultural Agnostic**: 'bread'

#### Exploiting the graph knowledge-based wikidata structure
- **Entity**: the item (e.g., "pizza")
- **P495**: country of origin
- **P2596**: culture
- **P172**: ethnic group
- **P37**: official language

In [ ]:
import requests
import json

In [2]:
url = 'https://query.wikidata.org/sparql'

Example of a query for extracting country of origin and culture for `Caponata(Q1034806)`

In [3]:
query = ''' 
SELECT ?origin ?culture
WHERE {
    wd:Q1034806 wdt:P495 ?origin;
                wdt:P2596 ?culture.
    SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],mul,en". }
}
'''

In [ ]:
r = requests.get(url, params = {'format': 'json', 'query': query})
data = r.json()

In [ ]:
properties = ['origin', 'culture']
query_result = {}
for prop in properties:
    query_result[prop] = data['results']['bindings'][0][prop]['value']

print(query_result)

# To transform the result in a json file

'''
    json_format = json.dumps(query_result)
    print(json_format)
'''

# Extracting the IDs

query_result2 = {}
for prop in properties:
    query_result2[prop] = data['results']['bindings'][0][prop]['value'].strip().split("/")[-1]

print(query_result2)

{'origin': 'http://www.wikidata.org/entity/Q38', 'culture': 'http://www.wikidata.org/entity/Q1460'}
{'origin': 'Q38', 'culture': 'Q1460'}
